<a href="https://colab.research.google.com/github/Khalidaman9555/IDS-AI/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# rt_sensor_simulator.py
# Modified to be importable and yield data for the dashboard.

import json
import random
import time

# Define sensor types and their typical normal/attack values
SENSOR_CONFIG = {
    "temperature_1": {"normal_range": (15, 30), "attack_value": 100, "unit": "°C"},
    "humidity_1": {"normal_range": (30, 60), "attack_value": 99, "unit": "%"},
    "pressure_1": {"normal_range": (1000, 1020), "attack_value": 500, "unit": "hPa"},
    "vibration_1": {"normal_range": (0, 0.5), "attack_value": 5.0, "unit": "g"},
    "plc_register_A": {"normal_range": (0, 100), "attack_value": 999, "unit": "value"},
    "flow_rate_1": {"normal_range": (50, 150), "attack_value": 0, "unit": "L/min"},
    "motor_current_1": {"normal_range": (1.0, 5.0), "attack_value": 20.0, "unit": "A"}
}

SENSORS = list(SENSOR_CONFIG.keys())

def generate_normal_sensor_reading(sensor_id):
    config = SENSOR_CONFIG[sensor_id]
    value = random.uniform(config["normal_range"][0], config["normal_range"][1])
    return {
        "timestamp": time.time(),
        "sensor_id": sensor_id,
        "value": round(value, 2),
        "unit": config["unit"],
        "status": "normal",
        "event_type": "sensor_reading" # Added for dashboard differentiation
    }

def generate_sensor_data_stream(num_events=10, anomaly_interval=0):
    """Yields a stream of sensor data events."""
    # anomaly_interval = 0 means no anomalies from sensor side for this basic version
    # Anomalies will be injected by the attack simulator primarily
    for i in range(num_events):
        sensor_id = random.choice(SENSORS)
        reading = generate_normal_sensor_reading(sensor_id)
        yield reading
        time.sleep(random.uniform(0.1, 0.5)) # Simulate some delay

if __name__ == "__main__":
    print("Running Real-time Sensor Simulator (example output):")
    for data_point in generate_sensor_data_stream(5):
        print(json.dumps(data_point))



Running Real-time Sensor Simulator (example output):
{"timestamp": 1746925749.490592, "sensor_id": "pressure_1", "value": 1011.3, "unit": "hPa", "status": "normal", "event_type": "sensor_reading"}
{"timestamp": 1746925749.6350584, "sensor_id": "vibration_1", "value": 0.3, "unit": "g", "status": "normal", "event_type": "sensor_reading"}
{"timestamp": 1746925749.7731547, "sensor_id": "flow_rate_1", "value": 94.89, "unit": "L/min", "status": "normal", "event_type": "sensor_reading"}
{"timestamp": 1746925750.106976, "sensor_id": "temperature_1", "value": 22.58, "unit": "\u00b0C", "status": "normal", "event_type": "sensor_reading"}
{"timestamp": 1746925750.4164183, "sensor_id": "motor_current_1", "value": 1.22, "unit": "A", "status": "normal", "event_type": "sensor_reading"}


In [ ]:
# rt_attack_simulator.py
# Modified to be importable and yield data for the dashboard.

import json
import random
import time

# Define attack types and their characteristics for simulation
ATTACK_TYPES = [
    {"name": "DDoS_TCP_SYN_Flood", "protocol": "TCP", "dst_port_range": (80, 80), "details": {"flags": "SYN", "payload_size": 0}},
    {"name": "Port_Scan_TCP", "protocol": "TCP", "dst_port_range": (1, 1024), "details": {"flags": "SYN"}},
    {"name": "Port_Scan_UDP", "protocol": "UDP", "dst_port_range": (1, 1024), "details": {}},
    {"name": "SQL_Injection", "protocol": "HTTP", "dst_port_range": (80, 80), "details": {"payload_pattern": "etc..."}},
    {"name": "DDoS_UDP_Flood", "protocol": "UDP", "dst_port_range": (53, 53), "details": {"payload_size": 64}},
    {"name": "ARP_Spoofing", "protocol": "ARP", "dst_port_range": (0,0), "details": {"message": "Gratuitous ARP reply"}}
    # More attack types can be added here
]

SOURCE_IPS = [f"192.168.1.{i}" for i in range(100, 150)] # Attacker IPs
TARGET_IPS = [f"10.0.0.{i}" for i in range(10, 15)]     # Victim IPs

def generate_attack_event():
    attack_config = random.choice(ATTACK_TYPES)
    src_ip = random.choice(SOURCE_IPS)
    dst_ip = random.choice(TARGET_IPS)
    dst_port = random.randint(attack_config["dst_port_range"][0], attack_config["dst_port_range"][1])
    if dst_port == 0 and attack_config["protocol"] != "ARP": # Port 0 is often invalid unless for specific cases
        dst_port = random.randint(1,65535) if attack_config["dst_port_range"] == (0,0) else dst_port

    event = {
        "timestamp": time.time(),
        "event_type": "attack_indicator", # Differentiate from sensor data
        "source_ip": src_ip,
        "destination_ip": dst_ip,
        "protocol": attack_config["protocol"],
        "src_port": random.randint(1025, 65535),
        "dst_port": dst_port,
        "details": {**attack_config["details"], "type": attack_config["name"]}
    }
    return event

def generate_attack_data_stream(num_events=5, event_interval_secs=1):
    """Yields a stream of attack data events."""
    for _ in range(num_events):
        yield generate_attack_event()
        time.sleep(event_interval_secs) # Simulate some delay between attack events

if __name__ == "__main__":
    print("Running Real-time Attack Simulator (example output):")
    for data_point in generate_attack_data_stream(3, event_interval_secs=0.5):
        print(json.dumps(data_point))



Running Real-time Attack Simulator (example output):
{"timestamp": 1746925762.9303756, "event_type": "attack_indicator", "source_ip": "192.168.1.105", "destination_ip": "10.0.0.10", "protocol": "HTTP", "src_port": 12031, "dst_port": 80, "details": {"payload_pattern": "etc...", "type": "SQL_Injection"}}
{"timestamp": 1746925763.4305596, "event_type": "attack_indicator", "source_ip": "192.168.1.102", "destination_ip": "10.0.0.14", "protocol": "ARP", "src_port": 27891, "dst_port": 0, "details": {"message": "Gratuitous ARP reply", "type": "ARP_Spoofing"}}
{"timestamp": 1746925763.9308681, "event_type": "attack_indicator", "source_ip": "192.168.1.132", "destination_ip": "10.0.0.13", "protocol": "TCP", "src_port": 63387, "dst_port": 80, "details": {"flags": "SYN", "payload_size": 0, "type": "DDoS_TCP_SYN_Flood"}}


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import time
import json
import os
import sys
from sklearn.preprocessing import StandardScaler

# Fix for Colab/notebook environments
try:
    __file__
except NameError:
    __file__ = os.getcwd()

# Add parent directory to path
current_dir = os.path.dirname(os.path.abspath(__file__))
project_root = os.path.dirname(current_dir)
testbed_sim_path = os.path.join(project_root, "testbed_simulation")
sys.path.append(testbed_sim_path)

try:
    from rt_sensor_simulator import generate_sensor_data_stream
    from rt_attack_simulator import generate_attack_data_stream
except ImportError as e:
    st.error(f"Could not import simulator modules: {e}")
    st.stop()

try:
    from tensorflow.keras.models import load_model
except ImportError:
    st.error("TensorFlow/Keras is not installed. Please install it to run the model.")
    st.stop()

# Initialize Streamlit
st.set_page_config(layout="wide", page_title="Real-time IDS Dashboard")
st.title("Real-time Intrusion Detection System Dashboard")

# --- Rest of your dashboard code with these fixes:
# 1. Change all "altain" to "altair"
# 2. Change "collums" to "columns"
# 3. Fix button syntax (colon after if statements)
# 4. Complete all dictionary definitions

2025-05-11 01:12:40.084 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 01:12:40.149 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-11 01:12:40.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 01:12:40.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 01:12:43.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 01:12:43.763 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 01:12:43.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
!pip install streamlit pyngrok
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦
added 22 packages in 855ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [ ]:
!pip install streamlit altair pydeck watchdog
!npm install localtunnel

import portpicker
import threading
import subprocess

def run_streamlit():
    port = portpicker.pick_unused_port()
    !streamlit run app.py --server.port {port} &>/content/logs.txt &

threading.Thread(target=run_streamlit, daemon=True).start()

!npx localtunnel --port {port}

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 3s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧⠙Usage: lt --port [num] <options>

Options:
  -p, --port                Internal HTTP server port                 [required]
  -h, --host                Upstream server providing forwarding
                                             [default: "https://localtunnel.me"]
  -s, --subdomain           Request this subdomain
  -l, --local-host          Tunnel traffic to this host instead of localhost,
                            override Host header to this host
      --local-https         Tunnel traffic to a local HTTPS server     [boolean]
      --local-cert          Path to certificate PEM file for local HTTPS server
      --local-key           Path to certificate key file for local HTTPS server
      --local-ca            Path to certificate authority file for self-signed
                            certificates
      --allow-invalid-cert  Disable certificate checks f

In [ ]:
# 1. Install required packages
!pip install streamlit pyngrok altair pandas numpy

# 2. Set up ngrok authtoken (replace with your actual token)
!ngrok config add-authtoken 2wve0ci4M0ajMTd9V6nEunsLP2b_25RXi3VihFCt5BfB8SpAU

# 3. Create the Streamlit app file
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime

# Your dashboard code here
st.title("Real-time IDS Dashboard")

# Sample data visualization
data = pd.DataFrame({
    'timestamp': [datetime.now()],
    'event_type': ['sample'],
    'value': [42]
})

st.line_chart(data.set_index('timestamp'))

# Add more of your dashboard components here...

# 4. Run Streamlit in the background
import threading
import os
import time
from pyngrok import ngrok

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.headless true')

thread = threading.Thread(target=run_streamlit)
thread.start()

# Wait for Streamlit to start
time.sleep(5)

# 5. Create ngrok tunnel
public_url = ngrok.connect(port='8501')
print(f"Your Streamlit app is running at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


UsageError: Line magic function `%%writefile` not found.


In [ ]:
!pip install pyngrok
!ngrok config add-authtoken 2wve0ci4M0ajMTd9V6nEunsLP2b_25RXi3VihFCt5BfB8SpAU

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
# 1. First verify and clean up existing ngrok configurations
!rm -rf /root/.config/ngrok
!mkdir -p /root/.config/ngrok

# 2. Set up your ngrok authtoken properly (replace with your actual token)
!ngrok config add-authtoken 2wve0ci4M0ajMTd9V6nEunsLP2b_25RXi3VihFCt5BfB8SpAU

# 3. Create the Streamlit app file
import os
app_content = """
import streamlit as st
import pandas as pd
import numpy as np
import time
from datetime import datetime

st.set_page_config(layout="wide")
st.title("Real-time IDS Dashboard")

status = st.empty()
chart = st.line_chart()

for i in range(100):
    data = pd.DataFrame({
        'timestamp': [datetime.now()],
        'value': [np.random.rand()]
    })
    status.write(f"Last update: {datetime.now().strftime('%H:%M:%S')}")
    chart.add_rows(data)
    time.sleep(1)
"""

with open('app.py', 'w') as f:
    f.write(app_content)

# 4. Run Streamlit with proper ngrok configuration
import threading
import time
from pyngrok import ngrok

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.headless true')

# Start Streamlit in background
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(5)  # Wait for Streamlit to initialize

# Create ngrok tunnel with proper configuration
try:
    # Explicitly specify protocol as http
    public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
    print(f"\n✅ Dashboard is live at: {public_url}\n")
    print(f"🔗 Alternative URL: {public_url.replace('https://', 'http://')}")

except Exception as e:
    print(f"\n❌ Error creating ngrok tunnel: {e}")
    print("\nTrying alternative Colab proxy method...")
    from google.colab.output import eval_js
    print(f"\n📊 Access your dashboard at: {eval_js('google.colab.kernel.proxyPort(8501)')}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml

✅ Dashboard is live at: NgrokTunnel: "https://946d-34-60-78-243.ngrok-free.app" -> "http://localhost:8501"


❌ Error creating ngrok tunnel: 'NgrokTunnel' object has no attribute 'replace'

Trying alternative Colab proxy method...

📊 Access your dashboard at: https://8501-m-s-3gwk157b4dt88-a.us-central1-0.prod.colab.dev


In [ ]:
# 1. First verify and clean up existing ngrok configurations
!rm -rf /root/.config/ngrok
!mkdir -p /root/.config/ngrok

# 2. Set up your ngrok authtoken properly (replace with your actual token)
!ngrok config add-authtoken YOUR_AUTH_TOKEN_HERE

# 3. Create the enhanced Streamlit app file
import os
app_content = """
import streamlit as st
import pandas as pd
import numpy as np
import time
from datetime import datetime
import altair as alt
from collections import defaultdict

# Initialize session state for attack tracking
if 'attack_data' not in st.session_state:
    st.session_state.attack_data = defaultdict(int)
    st.session_state.event_log = []
    st.session_state.detection_stats = {'normal': 0, 'attack': 0}

st.set_page_config(layout="wide")
st.title("🚨 Real-time IDS Dashboard")

# Dashboard columns
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("Event Stream")
    event_chart = st.empty()
    event_table = st.empty()

with col2:
    st.subheader("Detection Statistics")
    stats_placeholder = st.empty()
    dist_placeholder = st.empty()

# Simulate real-time events
for i in range(100):
    # Generate random event (80% normal, 20% attack)
    is_attack = np.random.random() < 0.2
    event_type = "Attack" if is_attack else "Normal"
    protocol = np.random.choice(["HTTP", "TCP", "UDP", "ARP"])
    source_ip = f"192.168.1.{np.random.randint(1, 255)}"

    # Update stats
    st.session_state.attack_data[protocol] += is_attack
    st.session_state.detection_stats['attack' if is_attack else 'normal'] += 1

    # Create event record
    event = {
        "timestamp": datetime.now().strftime('%H:%M:%S'),
        "type": event_type,
        "protocol": protocol,
        "source_ip": source_ip,
        "value": np.random.rand() * (10 if is_attack else 1)
    }
    st.session_state.event_log.append(event)

    # Keep only last 20 events
    if len(st.session_state.event_log) > 20:
        st.session_state.event_log.pop(0)

    # Update visualizations
    df = pd.DataFrame(st.session_state.event_log)

    with event_chart.container():
        chart = alt.Chart(df).mark_line().encode(
            x='timestamp:T',
            y='value:Q',
            color='type:N',
            tooltip=['type', 'protocol', 'source_ip']
        ).properties(width=600, height=300)
        st.altair_chart(chart)

    with event_table.container():
        st.dataframe(df.tail(5))

    with stats_placeholder.container():
        st.metric("Total Events", sum(st.session_state.detection_stats.values()))
        st.metric("Attack Events", st.session_state.detection_stats['attack'])
        st.metric("Normal Events", st.session_state.detection_stats['normal'])

    with dist_placeholder.container():
        attack_df = pd.DataFrame({
            'Protocol': list(st.session_state.attack_data.keys()),
            'Count': list(st.session_state.attack_data.values())
        })
        bar_chart = alt.Chart(attack_df).mark_bar().encode(
            x='Protocol:N',
            y='Count:Q',
            color='Protocol:N'
        ).properties(width=300, height=200)
        st.altair_chart(bar_chart)

    time.sleep(1)  # Simulate real-time updates
"""

with open('app.py', 'w') as f:
    f.write(app_content)

# 4. Run Streamlit with proper ngrok configuration
import threading
import time
from pyngrok import ngrok

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.headless true')

# Start Streamlit in background
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(5)  # Wait for Streamlit to initialize

# Create ngrok tunnel with proper configuration
try:
    public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
    print(f"\n✅ Dashboard is live at: {public_url}\n")
except Exception as e:
    print(f"\n❌ Error creating ngrok tunnel: {e}")
    from google.colab.output import eval_js
    print(f"\n📊 Access your dashboard at: {eval_js('google.colab.kernel.proxyPort(8501)')}")

In [ ]:
# 1. First verify and clean up existing ngrok configurations
!rm -rf /root/.config/ngrok
!mkdir -p /root/.config/ngrok

# 2. Set up your ngrok authtoken properly (replace with your actual token)
!ngrok config add-authtoken 2wve0ci4M0ajMTd9V6nEunsLP2b_25RXi3VihFCt5BfB8SpAU

# 3. Install required packages for Edge-IIoTset processing
!pip install streamlit pyngrok scikit-learn pandas numpy altair matplotlib

# 4. Create the enhanced Streamlit app file for Edge-IIoTset
import os
app_content = """
import streamlit as st
import pandas as pd
import numpy as np
import time
from datetime import datetime
import altair as alt
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Edge-IIoTset specific configuration
FEATURE_COLUMNS = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count'
]

ATTACK_CATEGORIES = {
    'Normal': ['Benign'],
    'DDoS': ['DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP'],
    'Scanning': ['Port_Scanning', 'OS_Scanning'],
    'Injection': ['SQL_injection', 'Command_injection'],
    'Malware': ['Backdoor', 'Ransomware', 'XSS']
}

# Initialize session state
if 'data' not in st.session_state:
    st.session_state.data = {
        'events': [],
        'attack_stats': defaultdict(int),
        'feature_stats': pd.DataFrame(columns=FEATURE_COLUMNS),
        'current_prediction': None,
        'model_accuracy': 0.0
    }
    # Simulate model accuracy (replace with actual model loading)
    st.session_state.data['model_accuracy'] = 0.982  # Example accuracy from Edge-IIoTset paper

# Load sample data (in production, replace with actual Edge-IIoTset loading)
def generate_edgeiiot_sample():
    protocols = ['TCP', 'UDP', 'HTTP', 'ICMP', 'Modbus', 'MQTT']
    attack_type = np.random.choice(list(ATTACK_CATEGORIES.keys()))
    subtype = np.random.choice(ATTACK_CATEGORIES[attack_type])

    sample = {
        'timestamp': datetime.now().strftime('%H:%M:%S.%f')[:-3],
        'attack_type': attack_type,
        'sub_type': subtype,
        'is_attack': 1 if attack_type != 'Normal' else 0,
        'protocol': np.random.choice(protocols),
        'source_ip': f"{np.random.randint(1,255)}.{np.random.randint(1,255)}.{np.random.randint(1,255)}.{np.random.randint(1,255)}",
        'dest_ip': f"10.0.{np.random.randint(1,50)}.{np.random.randint(1,255)}"
    }

    # Generate realistic feature values based on Edge-IIoTset characteristics
    features = {
        'flow_duration': np.random.uniform(0, 1000) if attack_type == 'Normal' else np.random.uniform(1000, 10000),
        'Header_Length': np.random.randint(20, 40) if attack_type == 'Normal' else np.random.randint(100, 1000),
        'Protocol Type': protocols.index(sample['protocol']),
        'Duration': np.random.uniform(0, 1) if attack_type == 'Normal' else np.random.uniform(1, 10),
        'Rate': np.random.uniform(0, 100) if attack_type == 'Normal' else np.random.uniform(100, 1000),
        'Srate': np.random.uniform(0, 50) if attack_type == 'Normal' else np.random.uniform(50, 500),
        'Drate': np.random.uniform(0, 50) if attack_type == 'Normal' else np.random.uniform(50, 500),
        'fin_flag_number': np.random.randint(0, 1) if attack_type == 'Normal' else np.random.randint(1, 10),
        'syn_flag_number': np.random.randint(0, 1) if attack_type == 'Normal' else np.random.randint(1, 100),
        'rst_flag_number': np.random.randint(0, 1) if attack_type == 'Normal' else np.random.randint(1, 10),
        'psh_flag_number': np.random.randint(0, 1) if attack_type == 'Normal' else np.random.randint(1, 5),
        'ack_flag_number': np.random.randint(0, 2) if attack_type == 'Normal' else np.random.randint(2, 100),
        'ece_flag_number': np.random.randint(0, 1),
        'cwr_flag_number': np.random.randint(0, 1),
        'ack_count': np.random.randint(0, 2) if attack_type == 'Normal' else np.random.randint(2, 50),
        'syn_count': np.random.randint(0, 1) if attack_type == 'Normal' else np.random.randint(1, 100),
        'fin_count': np.random.randint(0, 1) if attack_type == 'Normal' else np.random.randint(1, 10),
        'urg_count': np.random.randint(0, 1),
        'rst_count': np.random.randint(0, 1) if attack_type == 'Normal' else np.random.randint(1, 10)
    }

    sample.update(features)
    return sample

st.set_page_config(layout="wide")
st.title("🛡️ Edge-IIoTset Cybersecurity Dashboard")

# Dashboard layout
col1, col2 = st.columns([3, 1])

with col1:
    st.subheader("Real-time Network Traffic")
    event_chart = st.empty()
    st.subheader("Feature Correlation Heatmap")
    heatmap_placeholder = st.empty()

with col2:
    st.subheader("Threat Detection Statistics")
    stats_placeholder = st.empty()
    st.subheader("Attack Type Distribution")
    attack_placeholder = st.empty()
    st.subheader("Protocol Analysis")
    protocol_placeholder = st.empty()

# Simulation controls
attack_prob = st.sidebar.slider("Attack Probability", 0.1, 0.5, 0.25)
update_interval = st.sidebar.slider("Update Interval (ms)", 100, 2000, 500)
num_samples = st.sidebar.slider("Samples to Display", 50, 500, 100)

# Simulate real-time Edge-IIoTset data
for i in range(num_samples):
    sample = generate_edgeiiot_sample()

    # Update stats
    st.session_state.data['attack_stats'][sample['attack_type']] += 1
    st.session_state.data['events'].append(sample)
    if len(st.session_state.data['events']) > num_samples:
        st.session_state.data['events'].pop(0)

    # Update feature stats for correlation
    feature_row = {k: sample[k] for k in FEATURE_COLUMNS}
    st.session_state.data['feature_stats'] = pd.concat([
        st.session_state.data['feature_stats'],
        pd.DataFrame([feature_row])
    ], ignore_index=True)
    if len(st.session_state.data['feature_stats']) > 100:
        st.session_state.data['feature_stats'] = st.session_state.data['feature_stats'].iloc[1:]

    # Update visualizations
    df = pd.DataFrame(st.session_state.data['events'])

    with event_chart.container():
        # Plot key network features
        plot_df = df.melt(id_vars=['timestamp'],
                         value_vars=['flow_duration', 'Rate', 'Srate', 'Drate'],
                         var_name='metric', value_name='value')

        chart = alt.Chart(plot_df).mark_line().encode(
            x='timestamp:T',
            y=alt.Y('value:Q', scale=alt.Scale(zero=False)),
            color='metric:N',
            strokeDash='metric:N',
            tooltip=['timestamp', 'metric', 'value']
        ).properties(width=700, height=300)
        st.altair_chart(chart)

    with heatmap_placeholder.container():
        # Show feature correlation
        corr = st.session_state.data['feature_stats'].corr()
        fig, ax = plt.subplots(figsize=(8, 6))
        cax = ax.matshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
        fig.colorbar(cax)
        plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
        plt.yticks(range(len(corr.columns)), corr.columns)
        st.pyplot(fig)

    with stats_placeholder.container():
        total = len(df)
        attacks = len(df[df['is_attack'] == 1])
        st.metric("Total Samples", total)
        st.metric("Attack Samples", attacks, delta=f"{attacks/total:.1%}")
        st.metric("Model Accuracy", f"{st.session_state.data['model_accuracy']:.1%}")

    with attack_placeholder.container():
        attack_df = pd.DataFrame({
            'Type': list(st.session_state.data['attack_stats'].keys()),
            'Count': list(st.session_state.data['attack_stats'].values())
        })
        if not attack_df.empty:
            st.altair_chart(alt.Chart(attack_df).mark_bar().encode(
                x='Count:Q',
                y=alt.Y('Type:N', sort='-x'),
                color=alt.Color('Type:N', scale=alt.Scale(
                    domain=['Normal', 'DDoS', 'Scanning', 'Injection', 'Malware'],
                    range=['green', 'red', 'orange', 'purple', 'brown']))
            ).properties(width=300, height=200))

    with protocol_placeholder.container():
        if not df.empty:
            protocol_chart = alt.Chart(df).mark_arc().encode(
                theta='count()',
                color='protocol:N',
                tooltip=['protocol', 'count()']
            ).properties(width=200, height=200)
            st.altair_chart(protocol_chart)

    time.sleep(update_interval / 1000)  # Convert ms to seconds

# Add model evaluation section
st.sidebar.markdown("## Model Evaluation")
st.sidebar.write(f"Current accuracy: {st.session_state.data['model_accuracy']:.1%}")
st.sidebar.button("Retrain Model")  # Placeholder for actual retraining function
"""

with open('app.py', 'w') as f:
    f.write(app_content)

# 5. Run Streamlit with proper ngrok configuration
import threading
import time
from pyngrok import ngrok

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.headless true')

# Start Streamlit in background
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(5)  # Wait for Streamlit to initialize

# Create ngrok tunnel with proper configuration
try:
    public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
    print(f"\n✅ Dashboard is live at: {public_url}\n")
except Exception as e:
    print(f"\n❌ Error creating ngrok tunnel: {e}")
    from google.colab.output import eval_js
    print(f"\n📊 Access your dashboard at: {eval_js('google.colab.kernel.proxyPort(8501)')}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml

✅ Dashboard is live at: NgrokTunnel: "https://60de-34-60-78-243.ngrok-free.app" -> "http://localhost:8501"



In [ ]:
# 1. First verify and clean up existing ngrok configurations
!rm -rf /root/.config/ngrok
!mkdir -p /root/.config/ngrok

# 2. Set up your ngrok authtoken
!ngrok config add-authtoken 2wve0ci4M0ajMTd9V6nEunsLP2b_25RXi3VihFCt5BfB8SpAU


# 4. Create the enhanced Streamlit app file with CSV data pipeline
import os
app_content = """
import streamlit as st
import pandas as pd
import numpy as np
import time
from datetime import datetime
import altair as alt
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Configuration
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/lstm_model.h5"
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/datasets/ML-EdgeIIoT-dataset.csv"

# Edge-IIoTset feature columns (must match your model's training setup)
FEATURE_COLUMNS = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count'
]

# Initialize session state
if 'data' not in st.session_state:
    st.session_state.data = {
        'events': [],
        'predictions': [],
        'attack_stats': defaultdict(int),
        'feature_stats': pd.DataFrame(columns=FEATURE_COLUMNS),
        'model': None,
        'scaler': MinMaxScaler(),
        'data_iterator': None
    }

# Load LSTM model
@st.cache_resource
def load_ml_model():
    try:
        model = load_model(MODEL_PATH)
        st.success("✅ Model loaded successfully")
        return model
    except Exception as e:
        st.error(f"❌ Model loading failed: {str(e)}")
        st.stop()

st.session_state.data['model'] = load_ml_model()

# Data Pipeline
class EdgeIIoTDataIterator:
    def __init__(self, file_path):
        # Load dataset with proper types to reduce memory
        dtype = {col: np.float32 for col in FEATURE_COLUMNS}
        dtype.update({
            'label': 'category',
            'protocol_type': 'category',
            'src_ip': 'string',
            'dst_ip': 'string'
        })

        # Read in chunks if dataset is large
        self.df = pd.read_csv(file_path, dtype=dtype)
        self.current_idx = 0

        # Preprocess (adjust based on your CSV structure)
        if 'label' in self.df.columns:
            self.df['is_attack'] = self.df['label'].apply(
                lambda x: 0 if 'Normal' in str(x) else 1)

        # Ensure all feature columns exist
        for col in FEATURE_COLUMNS:
            if col not in self.df.columns:
                st.warning(f"Column {col} not found in dataset - filling with zeros")
                self.df[col] = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_idx >= len(self.df):
            self.current_idx = 0  # Loop dataset
            st.warning("⚠️ Reached end of dataset - looping from start")

        row = self.df.iloc[self.current_idx]
        self.current_idx += 1

        # Create sample with proper feature ordering
        sample = {
            'timestamp': datetime.now().strftime('%H:%M:%S.%f')[:-3],
            'is_attack': row.get('is_attack', 0),
            'protocol': str(row.get('protocol_type', 'TCP')),
            'source_ip': str(row.get('src_ip', '192.168.1.1')),
            'dest_ip': str(row.get('dst_ip', '10.0.0.1')),
            **{col: float(row[col]) for col in FEATURE_COLUMNS}
        }
        return sample

# Initialize data iterator
try:
    st.session_state.data['data_iterator'] = EdgeIIoTDataIterator(DATA_PATH)
    st.sidebar.success(f"Loaded dataset with {len(st.session_state.data['data_iterator'].df)} samples")
except Exception as e:
    st.sidebar.error(f"Failed to load dataset: {str(e)}")
    st.stop()

# Preprocessing function
def preprocess_for_model(sample):
    features = np.array([[sample[col] for col in FEATURE_COLUMNS]])

    # Scale features (fit on first call)
    if not hasattr(st.session_state.data['scaler'], 'n_samples_seen_'):
        st.session_state.data['scaler'].fit(features)

    scaled_features = st.session_state.data['scaler'].transform(features)
    return scaled_features.reshape(1, 1, len(FEATURE_COLUMNS))

# Dashboard UI
st.set_page_config(layout="wide")
st.title("🛡️ Edge-IIoTset IDS with LSTM Detection")

# Layout
col1, col2 = st.columns([3, 1])

with col1:
    st.subheader("Real-time Traffic & Predictions")
    event_chart = st.empty()
    st.subheader("Model Confidence Scores")
    confidence_plot = st.empty()

with col2:
    st.subheader("Detection Statistics")
    stats_placeholder = st.empty()
    st.subheader("Attack Probability")
    prob_placeholder = st.empty()
    st.subheader("Feature Values")
    feature_placeholder = st.empty()

# Controls
st.sidebar.subheader("Simulation Controls")
update_interval = st.sidebar.slider("Update Interval (ms)", 100, 2000, 500)
num_samples = st.sidebar.slider("Samples to Display", 50, 500, 100)
show_features = st.sidebar.multiselect(
    "Features to Display",
    FEATURE_COLUMNS,
    default=['flow_duration', 'Rate', 'syn_flag_number']
)

# Main processing loop
for i in range(num_samples):
    try:
        # Get next sample from dataset
        sample = next(st.session_state.data['data_iterator'])

        # Make prediction
        model_input = preprocess_for_model(sample)
        prediction = st.session_state.data['model'].predict(model_input, verbose=0)
        sample['prediction'] = float(prediction[0][0])
        sample['predicted_class'] = int(prediction > 0.5)

        # Update stats
        st.session_state.data['attack_stats']['Actual Attacks' if sample['is_attack'] else 'Actual Normal'] += 1
        st.session_state.data['attack_stats']['Detected Attacks' if sample['predicted_class'] else 'Detected Normal'] += 1
        st.session_state.data['events'].append(sample)
        st.session_state.data['predictions'].append(sample['prediction'])

        if len(st.session_state.data['events']) > num_samples:
            st.session_state.data['events'].pop(0)
            st.session_state.data['predictions'].pop(0)

        # Update visualizations
        df = pd.DataFrame(st.session_state.data['events'])

        with event_chart.container():
            # Plot selected features
            plot_df = df.melt(id_vars=['timestamp', 'is_attack'],
                            value_vars=show_features,
                            var_name='metric',
                            value_name='value')

            chart = alt.Chart(plot_df).mark_line().encode(
                x='timestamp:T',
                y=alt.Y('value:Q', scale=alt.Scale(zero=False)),
                color='metric:N',
                strokeDash='metric:N',
                tooltip=['timestamp', 'metric', 'value', 'is_attack']
            ).properties(width=700, height=300)

            # Add attack indicators
            attack_points = alt.Chart(df[df['is_attack'] == 1]).mark_point(
                size=100, filled=True, color='red', opacity=0.5
            ).encode(
                x='timestamp:T',
                y=alt.value(50)
            )

            st.altair_chart(chart + attack_points)

        # [Rest of your visualization code remains the same...]
        # (Include all the previous confidence plots, stats, etc.)

    except Exception as e:
        st.error(f"Error processing data: {str(e)}")
        break

    time.sleep(update_interval / 1000)

# Add dataset info
st.sidebar.subheader("Dataset Info")
st.sidebar.write(f"Total samples: {len(st.session_state.data['data_iterator'].df)}")
st.sidebar.write(f"Attack ratio: {st.session_state.data['data_iterator'].df['is_attack'].mean():.1%}")
"""

with open('app.py', 'w') as f:
    f.write(app_content)

# 5. Run Streamlit with proper ngrok configuration
import threading
import time
from pyngrok import ngrok

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.headless true')

# Start Streamlit in background
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()
time.sleep(5)  # Wait for Streamlit to initialize

# Create ngrok tunnel
try:
    public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
    print(f"\n✅ Dashboard is live at: {public_url}\n")
except Exception as e:
    print(f"\n❌ Error creating ngrok tunnel: {e}")
    from google.colab.output import eval_js
    print(f"\n📊 Access at: {eval_js('google.colab.kernel.proxyPort(8501)')}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml

✅ Dashboard is live at: NgrokTunnel: "https://9583-34-60-78-243.ngrok-free.app" -> "http://localhost:8501"



In [ ]:
# 1. First clean up existing processes
!pkill -f streamlit
!pkill ngrok

# 2. Create the corrected Streamlit app file
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import time

# MUST be the first Streamlit command
st.set_page_config(layout="wide")

# Then load your model
try:
    model = load_model("/content/drive/MyDrive/Colab Notebooks/lstm_model.h5")
except Exception as e:
    st.error(f"Model loading failed: {str(e)}")
    st.stop()

# Load Edge-IIoTset data
@st.cache_data
def load_data():
    try:
        df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/ML-EdgeIIoT-dataset.csv")
        # Add your data preprocessing here
        return df
    except Exception as e:
        st.error(f"Data loading failed: {str(e)}")
        st.stop()

df = load_data()

# Dashboard Title (must come after set_page_config)
st.title("Edge-IIoTset Cybersecurity Dashboard")

# Create layout
col1, col2 = st.columns([3, 1])

with col1:
    st.subheader("Real-time Network Traffic")
    chart_placeholder = st.empty()

with col2:
    st.subheader("Threat Detection")
    stats_placeholder = st.empty()

# Simulation loop
for i in range(100):
    sample = df.sample(1).iloc[0]

    try:
        prediction = model.predict(np.array([sample.values]))[0][0]
    except Exception as e:
        st.error(f"Prediction failed: {str(e)}")
        break

    # Update displays
    with chart_placeholder:
        st.line_chart(df.iloc[:i+1]['flow_duration'])

    with stats_placeholder:
        st.metric("Attack Probability", f"{prediction:.2%}")
        st.metric("Current Status", "ATTACK" if prediction > 0.5 else "NORMAL")

    time.sleep(0.5)

# 3. Run Streamlit in background
import threading
import os

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.headless true')

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# 4. Get access URL
time.sleep(5)
from google.colab.output import eval_js
print(f"\n📊 Access your dashboard at: {eval_js('google.colab.kernel.proxyPort(8501)')}")

UsageError: Line magic function `%%writefile` not found.


In [ ]:
# First, kill all existing ngrok tunnels
!pkill ngrok
!rm -rf /root/.config/ngrok
!mkdir -p /root/.config/ngrok

# Then restart with your configuration
!ngrok config add-authtoken 2wve0ci4M0ajMTd9V6nEunsLP2b_25RXi3VihFCt5BfB8SpAU

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Run Streamlit in background
import threading
import os

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.headless true')

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# Get access URL
time.sleep(5)
from google.colab.output import eval_js
print(f"\n📊 Access your dashboard at: {eval_js('google.colab.kernel.proxyPort(8501)')}")


📊 Access your dashboard at: https://8501-m-s-3gwk157b4dt88-a.us-central1-0.prod.colab.dev
